In [8]:
import os
import zipfile
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa
import tempfile
import shutil
import re

In [9]:
# Define paths
base_zip_path = r"D:\Morescope\Satellite data-20240806T085324Z-001.zip"
temp_dir = tempfile.mkdtemp()
final_zip_path = r"D:\Morescope\Processed_Parquet_Files.zip"

In [10]:

# Define the naming convention regex
csv_regex = re.compile(r"20231117_120secMother_AllCountries_002_TQ-Results_\d{4}_059_Markup001\(full\)\.csv")


In [11]:
def extract_zip(zip_path, extract_to):
    """Extract a ZIP file to a specified directory."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        return zip_ref.namelist()

# Extract the main ZIP file
extracted_main_dir = os.path.join(temp_dir, 'Satellite data')
os.makedirs(extracted_main_dir, exist_ok=True)
extracted_files = extract_zip(base_zip_path, extracted_main_dir)

In [12]:

# Process each inner ZIP file
parquet_paths = []
for inner_zip_name in extracted_files:
    inner_zip_path = os.path.join(extracted_main_dir, inner_zip_name)
    
    # Extract the inner ZIP file
    with zipfile.ZipFile(inner_zip_path, 'r') as inner_zip_ref:
        inner_zip_dir = os.path.join(temp_dir, 'inner_zip')
        os.makedirs(inner_zip_dir, exist_ok=True)
        inner_zip_ref.extractall(inner_zip_dir)
        
        # Filter and process CSV files
        for file_name in os.listdir(inner_zip_dir):
            if csv_regex.match(file_name):
                file_path = os.path.join(inner_zip_dir, file_name)
                
                # Read CSV file into DataFrame
                df = pd.read_csv(file_path)
                
                # Convert DataFrame to Parquet format
                parquet_path = file_path.replace('.csv', '.parquet')
                table = pa.Table.from_pandas(df)
                pq.write_table(table, parquet_path)
                parquet_paths.append(parquet_path)
                
                # Remove the CSV file to save space
                os.remove(file_path)
        
        # Clean up inner zip directory
        shutil.rmtree(inner_zip_dir)

In [14]:

# Zip all Parquet files into a new ZIP file
with zipfile.ZipFile(final_zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for parquet_path in parquet_paths:
        arcname = os.path.basename(parquet_path)
        zipf.write(parquet_path, arcname=arcname)
        os.remove(parquet_path)

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'C:\\Users\\ROG\\AppData\\Local\\Temp\\tmp_3_5z_j0\\inner_zip\\20231117_120secMother_AllCountries_002_TQ-Results_2011_059_Markup001(full).parquet'

In [ ]:

# Clean up the temporary directory
shutil.rmtree(temp_dir)